## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from composed_trackers import Config, build_from_cfg, TRACKERS
from pathlib import Path

## Configuration

In [2]:
fn_config = Path('configs/example_00.yaml')
cfg = Config.fromfile(fn_config)

In [3]:
print(cfg.text)


tracker:
    type: ComposedTrackers
    trackers: [NeptuneTracker, SimpleTracker]
    name: 'Experiment  00 name'
    description: 'Example of using configuration file.'
    tags: ['examples', 'config']
    offline: True
    SimpleTracker:
        root_path: './logs'
        exp_id_template: 'EXAM00-{i:03}'
    NeptuneTracker:
        project: 'USER_NAME/PROJECT_NAME'

seed: 777

optimizer:
  type: torch.optim.Adam
  weight_decay: 0.0001
  lr: 0.6



In [4]:
# Comment this line if you finish debugging.
cfg.tracker.offline = True 

## Create tracker from config

In [5]:

tracker = build_from_cfg(cfg.tracker, TRACKERS, 
                         {'params': cfg.to_flatten_dict()}  # flattening config --> params
                        )

Neptune is running in offline mode. No data is being logged to Neptune.
Disable offline mode to log your experiments.


In [6]:
tracker.describe()

ComposedTrackers description:
  name        : Experiment  00 name
  description : Example of using configuration file.
  tags        : ['examples', 'config']
  offline     : True

NeptuneTracker
    exp_id: None
   project: USER_NAME/PROJECT_NAME

SimpleTracker
    exp_id: EXAM00-005
      path: logs/EXAM00-005



In [7]:
# all params of experiment
tracker.params

{'tracker.type': 'ComposedTrackers',
 'tracker.trackers': "['NeptuneTracker', 'SimpleTracker']",
 'tracker.name': 'Experiment  00 name',
 'tracker.description': 'Example of using configuration file.',
 'tracker.tags': "['examples', 'config']",
 'tracker.offline': True,
 'tracker.SimpleTracker.root_path': './logs',
 'tracker.SimpleTracker.exp_id_template': 'EXAM00-{i:03}',
 'tracker.NeptuneTracker.project': 'USER_NAME/PROJECT_NAME',
 'seed': 777,
 'optimizer.type': 'torch.optim.Adam',
 'optimizer.weight_decay': 0.0001,
 'optimizer.lr': 0.6}

## Use trackers

In [8]:
tracker.append_tag('introduction-minimal-example')

In [9]:
n = 117
for i in range(1, n):
    tracker.log_metric('iteration', i)
    tracker.log_metric('loss', 1/i**0.5)
    tracker.log_text('magic values', 'magic value {}'.format(0.95*i**2))
tracker.set_property('n_iterations', n)


tracker.log_text_as_artifact('Hello', 'summary.txt')

## Stop

In [10]:
tracker.stop()

NeptuneTracker stopping...  Ok.
BaseTracker stopping... Ok.


In [11]:
tracker.describe(ids_only=True)

NeptuneTracker : None
SimpleTracker  : EXAM00-005



In [12]:
!ls {tracker.path} -1

artifacts
metrics.csv
metrics.json
params.yaml
properties.yaml
tags.yaml
texts.csv
texts.json
